First we import some methods.

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from numpy import array
from time import perf_counter
from Complexes.VR.Expansion.ExpansionBruteForce import ExpansionBruteForce
from Complexes.VR.Expansion.Incremental import Incremental
from Complexes.VR.Expansion.Inductive import Inductive
from Complexes.VR.Skeleton.SkeletonBruteForce import *
from random import random
from typing import ClassVar, Dict
from numpy import pi, sin, cos, arange
from plotnine import ggplot, aes, geom_line, labs, theme, xlab, ylab
from IPython.display import clear_output
from Complexes.VR.Skeleton.Sklearn import Sklearn
import pandas as pd

A progress bar will allow us to track progress on long running code.

In [2]:
def progress_bar(progress: float, bar_count: int = 80):
    clear_output(wait=True)
    progress_bar_count = int(progress * bar_count)
    print(f"[{'+' * progress_bar_count}{' ' * (bar_count - progress_bar_count)}]")

We define the Euclidean metric.

In [3]:
def d(x: ndarray, y: ndarray) -> float:
    if x.size != y.size:
        raise ValueError("Sizes of x and y do not match.")
    total = 0
    for xx, yy in zip(x, y):
        total += (xx - yy) ** 2
    return total ** 0.5

We define two sampling methods for generating datasets.

In [4]:
def random_points(point_count: int) -> List[ndarray]:
    points = []
    for _ in range(point_count):
        points.append(array([random(), random(), random()]))
    return points


def sample_noisy_circle(point_count: int, radius: float = 1, radius_noise: float = None) -> List[ndarray]:
    if radius_noise is None:
        radius_noise = radius / 10
    points = []
    for _ in range(point_count):
        r = radius + ((((random() ** 0.5) * 2) - 1) * radius_noise)
        angle = 2 * pi * random()
        points.append(array([r * cos(angle), r * sin(angle)]))
    return points

We use these functions to define some datasets for later.

In [5]:
@dataclass
class TestSet:
    label: str
    desc: str
    data: List[ndarray]


# Random test sets
test_sets: Dict[str, TestSet] = dict()
for n in [10, 100, 1000, 10000, 100000]:
    la = f"random_{n}"
    de = f"{n} random points taken uniformly at random from [0,1]^3."
    da = random_points(n)
    t = TestSet(la, de, da)
    test_sets[la] = t

# Noisy circle test sets
for n in [1000, 10000, 100000]:
    la = f"noisy_unit_circle_{n}"
    de = f"{n} random points taken uniformly from an annulus."
    da = sample_noisy_circle(n)
    t = TestSet(la, de, da)
    test_sets[la] = t

We're moving towards a nice function to run our different methods, first a dataclass.

In [6]:
@dataclass
class VRRun:
    test_set_label: str = None
    test_set_size: int = None
    epsilon: float = None
    max_dim: int = None
    skeleton_method: str = None
    expansion_method: str = None
    skeleton_time: float = None
    expansion_time: float = None

    def __init__(self):
        pass

And the mentioned function (this will save a lot of time).

In [7]:
def do_vr_run(run_class: ClassVar, test_set: TestSet, max_dim: int, epsilon: float) -> VRRun:
    run = VRRun()
    run.test_set_label = test_set.label
    run.test_set_size = len(test_set.data)
    run.epsilon = epsilon
    run.max_dim = max_dim

    for base in run_class.__bases__:
        if "compute_expansion" in base.__abstractmethods__:
            run.skeleton_method = base.__name__
        elif "compute_skeleton" in base.__abstractmethods__:
            run.expansion_method = base.__name__

    run_instance = run_class(test_set.data, epsilon, d)

    start = perf_counter()
    run_instance.compute_skeleton()
    end = perf_counter()
    run.skeleton_time = round(end - start, 3)

    start = perf_counter()
    run_instance.compute_expansion(max_dim)
    end = perf_counter()
    run.expansion_time = round(end - start, 3)

    return run

Time for our first test! We'll first just confirm that the skeleton method from Sklearn is better than brute force, which is expected.

In [8]:
class BruteForceSkeleton(SkeletonBruteForce, Incremental):
    pass


class SklearnSkeleton(Sklearn, Incremental):
    pass


skeleton_runs: Dict[str, List[VRRun]] = {
    "bf": [],
    "sk": [],
}

for n in range(100, 1000, 10):
    my_data = sample_noisy_circle(n)
    my_test_set = TestSet("", "", my_data)

    progress_bar((n / 1000))
    bf_instance = do_vr_run(BruteForceSkeleton, my_test_set, 3, 0.01)
    sk_instance = do_vr_run(SklearnSkeleton, my_test_set, 3, 0.01)

    skeleton_runs["bf"].append(bf_instance)
    skeleton_runs["sk"].append(sk_instance)

clear_output(wait=True)
print("Complete.")

Complete.


Now we plot the results.

In [19]:
x = [run.test_set_size for run in skeleton_runs["bf"]]
y1 = [run.skeleton_time for run in skeleton_runs["bf"]]
y2 = [run.skeleton_time for run in skeleton_runs["sk"]]

df = pd.DataFrame(zip(x, y1, y2), columns=["Point count", "Brute force", "Sklearn"])
df = pd.melt(df, id_vars="Point count")

p = (
        ggplot(df)
        + aes(x="Point count")
        + geom_line(aes(y="value", linetype="variable"))
        + labs(linetype="Skeleton method")
        + xlab("Point count")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
)

In [20]:
p.save("1-skeleton-methods.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 1-skeleton-methods.pdf


This clearly reaffirms what we believed! Now we move onto an investiagtion into expansion methods

In [11]:
# COMPARING SKELETON METHODS

class BruteForceExpansion(Sklearn, ExpansionBruteForce):
    pass


class InductiveExpansion(Sklearn, Inductive):
    pass


class IncrementalExpansion(Sklearn, Incremental):
    pass


initial_expansion_runs: Dict[str, List[VRRun]] = {
    "bf;nuc1000": [],
    "in;nuc1000": [],
    "ic;nuc1000": [],
}

max_dim = 10

for ep in arange(0.001, 0.0101, 0.0001):
    progress_bar((ep * 100) ** 6)
    bf_instance = do_vr_run(BruteForceExpansion, test_sets["noisy_unit_circle_1000"], max_dim, ep)
    in_instance = do_vr_run(InductiveExpansion, test_sets["noisy_unit_circle_1000"], max_dim, ep)
    ic_instance = do_vr_run(IncrementalExpansion, test_sets["noisy_unit_circle_1000"], max_dim, ep)

    initial_expansion_runs["bf;nuc1000"].append(bf_instance)
    initial_expansion_runs["in;nuc1000"].append(in_instance)
    initial_expansion_runs["ic;nuc1000"].append(ic_instance)

clear_output(wait=True)
print("Complete.")

Complete.


In [27]:
x = [run.epsilon for run in initial_expansion_runs["bf;nuc1000"]]
y1 = [run.expansion_time for run in initial_expansion_runs["bf;nuc1000"]]
y2 = [run.expansion_time for run in initial_expansion_runs["in;nuc1000"]]
y3 = [run.expansion_time for run in initial_expansion_runs["ic;nuc1000"]]

df = pd.DataFrame(zip(x, y1, y2, y3), columns=["Epsilon", "Brute force", "Inductive", "Incremental"])
df = pd.melt(df, id_vars="Epsilon")

p = (
        ggplot(df)
        + aes(x="Epsilon")
        + geom_line(aes(y="value", linetype="variable"))
        + labs(linetype="Expansion method")
        + xlab("$\\hat\\varepsilon$")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
)

In [28]:
p.save("1-initial-expansion-methods.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 1-initial-expansion-methods.pdf


So it seems that brute force is _very_ bad, as expected. Lets investigated incremental vs inductive a bit more.

In [23]:
expansion_runs: Dict[str, List[VRRun]] = {
    "in;nuc1000": [],
    "ic;nuc1000": [],
}

max_dim = 8

for ep in arange(0.01, 0.055, 0.001):
    progress_bar((ep * 20) ** 3)
    in_instance = do_vr_run(InductiveExpansion, test_sets["noisy_unit_circle_1000"], max_dim, ep)
    ic_instance = do_vr_run(IncrementalExpansion, test_sets["noisy_unit_circle_1000"], max_dim, ep)

    expansion_runs["in;nuc1000"].append(in_instance)
    expansion_runs["ic;nuc1000"].append(ic_instance)

clear_output(wait=True)
print("Complete.")

Complete.


In [29]:
x = [run.epsilon for run in expansion_runs["in;nuc1000"]]
y1 = [run.expansion_time for run in expansion_runs["in;nuc1000"]]
y2 = [run.expansion_time for run in expansion_runs["ic;nuc1000"]]

df = pd.DataFrame(zip(x, y1, y2), columns=["Epsilon", "Inductive", "Incremental"])
df = pd.melt(df, id_vars="Epsilon")

p = (
        ggplot(df)
        + aes(x="Epsilon")
        + geom_line(aes(y="value", linetype="variable"))
        + labs(linetype="Expansion method")
        + xlab("$\\hat\\varepsilon$")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
)

In [30]:
p.save("1-expansion-methods.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 1-expansion-methods.pdf
